In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [ ]:
import os, glob
from cryoant.daq.xia.listmode import load_and_process

import numpy as np
import pandas as pd
import cryoant as ct

import matplotlib.pyplot as plt
from plotly import express as px
from plotly import graph_objects as go

plt.style.use(f"{list(ct.__path__)[0]}/plot.mpl")

DPI_VIS, DPI_SV = 200, 50
DATE = "240812"

In [ ]:
df["ig_laser"] = np.abs(
    (df.time % 1 / 300) - (df.time % 1 / 300).rolling(1000).median()
) < 0.05 * np.mean(df.time % 1 / 300)
df["ig_laser"] = df.ig_laser & df.ig_event

In [ ]:
%%capture
%%
directory = "/beest_data/summer2024/Be7_Ta_PR_Mask_listmode_ben/d/"
files = glob.glob(os.path.join(directory, "*_Al_*/*.bin"))
# files = glob.glob(os.path.join(directory, "*.bin"))

# Sort files by filesize
files.sort(key=os.path.getsize)
# files.sort(key=lambda x: x.split("/")[-1])
files = [file for file in files if os.path.getsize(file) > 2e9]
files = [file for file in files if DATE in file]

dev_file = files[
    0
]  # for this file, which is currently 7-22 chunk34, the best channel for spectrum seems to be 28, and tpz is 250.5
dev_file

In [ ]:
df = pd.read_hdf("out/processed/240812.h5", key="data")

In [ ]:
!python -m cryoant.daq.xia.listmode dataset-processor "240812" -k kwfile.json

## Spectrum from single 2 GB listmode output file

In [ ]:
%%markdown
OMP_NUM_THREADS = os.environ["OMP_NUM_THREADS"] # export OMP_NUM_THREADS=1
OPENBLAS_NUM_THREADS = os.environ["OPENBLAS_NUM_THREADS"] # export OPENBLAS_NUM_THREADS=1
MKL_NUM_THREADS = os.environ["MKL_NUM_THREADS"] # export MKL_NUM_THREADS=1
VECLIB_MAXIMUM_THREADS = os.environ["VECLIB_MAXIMUM_THREADS"] # export VECLIB_MAXIMUM_THREADS=1
NUMEXPR_NUM_THREADS = os.environ["NUMEXPR_NUM_THREADS"] # export NUMEXPR_NUM_THREADS=1

In [ ]:
%%markdown
"""Set to Existing Values"""    
os.environ["OMP_NUM_THREADS"] = OMP_NUM_THREADS # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = OPENBLAS_NUM_THREADS # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = MKL_NUM_THREADS # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = VECLIB_MAXIMUM_THREADS # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = NUMEXPR_NUM_THREADS # export NUMEXPR_NUM_THREADS=1

In [ ]:
#: Load and Process a File
kwdict = dict(
    rise=450,
    flat=50,
    fast=50,
    start_tpz=400,
    max_tpz=8000,
    debug_plots=False,
    outdir="out",
)

# os.environ["OMP_NUM_THREADS"] = "1"  # export OMP_NUM_THREADS=1
# os.environ["OPENBLAS_NUM_THREADS"] = "1"  # export OPENBLAS_NUM_THREADS=1
# os.environ["MKL_NUM_THREADS"] = "1"  # export MKL_NUM_THREADS=1
# os.environ["VECLIB_MAXIMUM_THREADS"] = "1"  # export VECLIB_MAXIMUM_THREADS=1
# os.environ["NUMEXPR_NUM_THREADS"] = "1"  # export NUMEXPR_NUM_THREADS=1

dfs = []
df, header, opt_tpzs = load_and_process(
    files[-1], multithread=True, drop_trace=True, events_total=1e6, kwdict=kwdict
)
dfs.append(df)
for file in files[-5:-1]:
    df, _, _ = load_and_process(
        file,
        multithread=True,
        drop_trace=True,
        events_total=1e6,
        kwdict=kwdict,
        known_tpz=opt_tpzs,
    )
    dfs.append(df)

df = pd.concat(dfs)
df.to_hdf("out/processed/240812.h5", key="df", mode="w")

In [ ]:
files[-3].split("/")[-1]

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
import matplotlib_inline.backend_inline

_VSCode_matplotLib_FigureFormats = (
    matplotlib_inline.backend_inline.InlineBackend.instance().figure_formats
)
_VSCode_matplotLib_FigureFormats.add("svg")
matplotlib_inline.backend_inline.set_matplotlib_formats(
    *_VSCode_matplotLib_FigureFormats
)

In [ ]:
%%capture
%%
h, b = np.histogram(df.pulse_onset, bins=500)
fig = go.Figure(data=[go.Scatter(x=b[:-1], y=h, mode='lines', line=dict(shape='hv'))])
fig.update_layout(
    width=800,
    height=300,
    title="Pulse Onset Histogram",
    xaxis_title="Bins",
    yaxis_title="Counts"
)
fig.show()

In [ ]:
h, b = np.histogram(df.pulse_onset, bins=500)
plt.figure(figsize=(8, 3), dpi=DPI_SV)
plt.step(b[:-1], h, where="mid")
# plt.gca().set(xlim=(0, len(df.trace[0])))
display(plt.gcf())
plt.close()

## Ideal Filter Parameters

PZ is probably fine, but we need to try different shaping times

In [ ]:
for channel in df.channel.unique():
    h, b = np.histogram(
        df[(df.channel == channel)].height_mV,
        bins=2000,
        range=(0, 50),
    )
    fig, ax = plt.subplots(dpi=DPI_VIS)
    ax.step(b[:-1], h)
    ax.set(
        xlabel="Height",
        ylabel="Counts",
        title=f"Spectrum Channel {channel}",
        yscale="log",
        xlim=(0, 50),
    )
    display(fig)
    plt.close(fig)

In [ ]:
%%capture
%%
"""Shaping Validation
"""
from cryoant.daq.xia.listmode import shaping_tester, kwfilt

kwdict = dict(rise=400, flat=150, fast=50, start_tpz=300, end_tpz=1200, step=5)

for channel in df.channel.unique():
    col, heights, fast_amps, onsets = shaping_tester(
        df[(df.channel == channel) & (df.multiplicity > 5)],
        opt_tpzs,
        kwdict,
        **kwfilt(kwdict, shaping_tester)
    )

    h, b = np.histogram(heights, bins=500)
    fig, ax = plt.subplots()
    ax.step(b[:-1], h)
    ax.set(
        xlabel="Height",
        ylabel="Counts",
        title="Height Spectrum",
        yscale="log",
    )

In [ ]:
%%capture out
%%
from cryoant.daq.xia.listmode import get_pulse_height

for i in np.random.randint(0, len(df), 5):
    get_pulse_height(
        df.iloc[i].trace,
        **kwfilt(kwdict, get_pulse_height, tpz=opt_tpzs[df.iloc[i].channel]),
        debug_plot=True,
    )

OK so TPZ are really bad, let's debug that first:

#TODO The fact that purple and the green trapezoid with the red flat are not the same exact trapezoid is a real problem because these are simply the filtered result as done in the_filter and done manually using trapezoidalFilter. They should be the same. Why are they not????

In [ ]:
%%capture
%%
from cryoant.daq.xia.listmode import get_best_tpz

kwdict = kwdict | dict(start_tpz=500, end_tpz=4000, step=50)
display(kwdict)
for i in np.random.randint(0, len(df[df.height > 100]), 5):
    get_best_tpz(
        df[df.height > 100].iloc[i].trace,
        **kwfilt(kwdict, get_best_tpz),
        debug_plots=True,
        kwdict=kwdict
    )

## Plot Trigger Realtime

In [ ]:
"""Apply File Timestamp to Event Time"""

timestamps = df.fname.apply(lambda x: "_".join(x.split("_")[-3:-1]))
df["timestamp"] = pd.to_datetime(timestamps, format="%Y-%m-%d_%H.%M.%S")
df["realtime"] = pd.to_timedelta(df.time, unit="s") + df["timestamp"]

In [ ]:
"""Plot Realtime vs Height"""

for channel in df.channel.unique():
    fig, ax = plt.subplots(figsize=(8, 3), dpi=DPI_VIS, constrained_layout=True)
    for fname in df[df.channel == channel].fname.unique():
        data = df[(df.channel == channel) & (df.fname == fname)]
        ax.scatter(data.realtime, data.height_mV, s=0.3, alpha=0.4, lw=0, label=fname)
    ax.set(
        xlabel="Realtime",
        ylabel="Height (mV)",
        title=f"Channel {channel}",
        yscale="log",
    )
    ax.tick_params(axis="x", labelsize=8)
    display(fig)
    plt.close(fig)

# Diagnose Scatter Weirdness

No banding in scatter plot: what are the different regions?
- Regions were likely me not doing PZ correction by accident (and other bugs)
- Banding has been solved by getting at least 100k data points (before ~ 10k)

Don't forget that each channel will be significantly different, so definitely isolate each individual channel before plotting/analyzing.

What to keep?

Let's separate by time rather than each individual file.
There's four regions seen in the plots above.

In [ ]:
%%capture
%%
for channel in df.channel.unique():
    nfs = len(df[df.channel == channel].fname.unique())
    nrows = int(np.ceil(np.sqrt(nfs)))
    ncols = int(np.ceil(nfs / nrows))
    fig, axes = plt.subplots(nrows, ncols, figsize=(30, 30) , dpi=300, constrained_layout=True)
    for i, fname in enumerate(df[df.channel == channel].fname.unique()):
        ax = axes.flat[i]
        data = df[(df.channel == channel) & (df.fname == fname) & (df.multiplicity > 5)]
        ax.scatter(data.height_mV, data.otherV, s=0.3, alpha=0.4, lw=0, label=fname)
        ax.set(
            xlabel="Height [mV]",
            ylabel="Other [$_\\sum$mV]",
            title=f"{fname}",
        )
        ax.title.set_fontsize(8)
    # ax.scatter(
    #     df[df.channel == channel].height_mV,
    #     df[df.channel == channel].otherV,
    #     s=0.3,
    #     alpha=0.4,
    #     lw=0,
    # )
    fig.suptitle(
        "Channel {channel}",
    )
    display(fig)
    plt.close(fig)

For each time region, for each channel, plot height vs otherV

The four time regions are:
- Before 24-08-12 23:00
- Between 24-08-12 23:00 and 24-08-13 2:00
- Between 24-08-13 2:00 and 24-08-13 12:00
- After 24-08-13 12:00

The banding looks appropriate for each region. No need to perform cuts.

In [ ]:
%%capture
%%

time_regions = [
    ("Before 24-08-12 23:00", df.realtime < "2024-08-12 23:00"),
    ("Between 24-08-12 23:00 and 24-08-13 2:00", (df.realtime >= "2024-08-12 23:00") & (df.realtime < "2024-08-13 02:00")),
    ("Between 24-08-13 2:00 and 24-08-13 12:00", (df.realtime >= "2024-08-13 02:00") & (df.realtime < "2024-08-13 12:00")),
    ("After 24-08-13 12:00", df.realtime >= "2024-08-13 12:00"),
]
for channel in df.channel.unique():
    for region_name, region_df in time_regions:
        fig, ax = plt.subplots(figsize=(10, 6), dpi=DPI_VIS)
        data = df[(df.channel == channel) & region_df]
        ax.scatter(data.height_mV, data.otherV, s=0.3, alpha=0.4, lw=0)
        ax.set(
            xlabel="Height [mV]",
            ylabel="Other [$_\\sum$mV]",
            title=f"Channel {channel} - {region_name}",
            xlim=(0, 25),
            ylim=(0, 100),
        )
        display(fig)
        plt.close(fig)

Identify regions in scatter plot for plotting their traces.

In [ ]:
%%capture
%%

#: Low-Us High-Them region
#: Red
r1a, r1b, r1r = 100, 50, 5
#: Zero-Us High-Them Region
#: Green
r2a, r2b, r2r = 0, 50, 5
#: Low Both-Of-Us Region
#: Blue
r3a, r3b, r3r = 0, 20, 5
#: Odd Bands above Low BothOfUs Region
#: Yellow
# r4a, r4b, r4r = 20, 50, 5
#: Us Zero-Them Region
#: Purple
# r5a, r5b, r5r = 250, 5, 10
#: Normal High-Them Lobe
#: Cyan
# r6a, r6b, r6r = 1000, 600, 50
#: Higher-Us than Them Lobe
#: Magenta
# r7a, r7b, r7r = 2000, 900, 50


def circle(x, y, x1, y1, r):
    return (x - x1) ** 2 + (y - y1) ** 2 < r**2


df["region1"] = [circle(a, b, r1a, r1b, r1r) for a, b in zip(df.height, df.otherV)]
df["region2"] = [circle(a, b, r2a, r2b, r2r) for a, b in zip(df.height, df.otherV)]
df["region3"] = [circle(a, b, r3a, r3b, r3r) for a, b in zip(df.height, df.otherV)]
# df["region4"] = [circle(a, b, r4a, r4b, r4r) for a, b in zip(df.height, df.otherV)]
# df["region5"] = [circle(a, b, r5a, r5b, r5r) for a, b in zip(df.height, df.otherV)]
# df["region6"] = [circle(a, b, r6a, r6b, r6r) for a, b in zip(df.height, df.otherV)]
# df["region7"] = [circle(a, b, r7a, r7b, r7r) for a, b in zip(df.height, df.otherV)]

fig, ax = plt.subplots(figsize=(10, 10), dpi=DPI_SV)
colors = [
    "red",
    "green",
    "blue",
    # "yellow",
    # "purple",
    # "cyan",
    # "magenta"
]
rdesc = [
    "Low-Us High-Them",
    "Zero-Us High-Them",
    "Low Both-Of-Us",
    # "Odd Bands above Low Both-Of-Us",
    # "Us Zero-Them",
    # "Normal High-Them Lobe",
    # "Higher-Us than Them Lobe",
]
ax.scatter(df.height, df.otherV, s=0.1, alpha=0.2, lw=0)
for region in [
    "region1",
    "region2",
    "region3",
    # "region4",
    # "region5",
    # "region6",
    # "region7",
]:
    ax.scatter(
        df.height[df[region]],
        df.otherV[df[region]],
        s=0.1,
        alpha=0.5,
        c=colors.pop(0),
        lw=0,
        label=region,
    )

Plot Trace waveforms and Average Trace for Selected Regions

In [ ]:
%%capture
%%

for region in [
    "region1",
    "region2",
    "region3",
    # "region4",
    # "region5",
    # "region6",
    # "region7",
]:
    fig, ax = plt.subplots(figsize=(8, 2), dpi=200)
    [
        ax.plot(trc, lw=0.5, alpha=1)
        for trc in df.trace[df[region]].sample(frac=50 / 2000)
    ]
    ax.plot(
        np.mean(df.trace[df[region]], axis=0),
        lw=2,
        alpha=1,
        c="w",
    )
    ax.set(title=f"Region {region[-1]} Pulses (Us)\n{rdesc[int(region[-1]) - 1]}")

## Finally Apply Laser Heating Calibration

Turns out the multiplicity cut is bad: data with multiplicity down to even 2 contains some laser data. Cutting that off prematurely produces the shelf I've seen in a couple spectrum plots. Therefore it is optimal to treat all data as potentially laser data.

Luckily, a correction based on otherV won't affect nuclear data nearly at all because the otherV during a nuclear event will be zero.

In [ ]:
from beest.laser import correct_substrate_heating, deskewFFT, argmax_part
from beest.plot import ScatterHistogramPlot

In [ ]:
from joblib import Parallel, delayed

In [ ]:
def boop(df, channel):
    data = df[(df.channel == channel)]
    corrected_values = correct_substrate_heating(
        data.height_mV, data.sumV, dev_plot=True, dev_name=f"ch{channel}"
    )
    return data.index, corrected_values


results = Parallel(n_jobs=4)(
    delayed(boop)(df, channel) for channel in df.channel.unique()
)
# results = [boop(df, channel) for channel in df.channel.unique()]

df.drop("correct", inplace=True, axis=1)
results = [r for r in results if r is not None]
for indices, corrected_values in results:
    df.loc[indices, "correct"] = corrected_values

In [ ]:
df.to_hdf("out/processed/240812-corrected.h5", key="data", mode="w")

In [ ]:
for channel in df.channel.unique():
    data = df[(df.channel == channel)]
    xmin, xmax = np.percentile(data.correct, [1, 99])
    ymin, ymax = np.percentile(data.otherV, [1, 99])
    fig, axs = plt.subplots(1, 2, figsize=(10, 5), dpi=200, constrained_layout=True)
    axs[0].scatter(data.correct, data.otherV, s=0.1, alpha=0.5, lw=0)
    axs[0].set(
        xlabel="Corrected Height [mV]",
        ylabel="OtherV [$_\\sum$mV]",
        title=f"Channel {channel}",
        xlim=(xmin, xmax),
        ylim=(ymin, ymax),
    )
    h, b = np.histogram(data.correct, bins=5000)
    axs[1].step(b[:-1], h)
    axs[1].set(
        xlabel="Corrected Height [mV]",
        ylabel="Counts",
        title=f"Channel {channel}",
        yscale="log",
        xlim=(xmin, xmax),
    )

    display(fig)
    plt.close(fig)

## Debug Correction

I think the problem was manifold.
- The Al data is low amplitude so it's important to set the FFT parameters quite near the expected values
- I was using mV (> 1) vs V (0 < X < 1) which made the existing code unstable. Turns out bin number is exchangeable with bin width when FFTing values less than 1

The code is now extremely robust
- It uses actual frequencies rather than bin values to give the FFT frequency a meaning (1/mV) which is directly attributable to the peak spacing
- It uses percentiles to zone in on the data of import rather than hard coded ranges which differ per detector and probably per day.

In [ ]:
channel = 0
slice_max = 0.025
numLines = 20
firstLine = 7
dev_plot = True
dev_name = "dev"

# data = df[(df.channel == channel)]
data = df[(df.channel == channel) & (df.multiplicity > 5)]
correct = correct_substrate_heating(
    data.height_mV, data.sumV, dev_plot=True, dev_name=f"ch{channel}"
)

fig, ax = plt.subplots(figsize=(10, 6), dpi=DPI_VIS)
ax.scatter(
    correct,
    data.otherV,
    s=0.3,
    alpha=0.4,
    lw=0,
)
ax.set(
    xlabel="Height [mV]",
    ylabel="Sum [$_\\sum$mV]",
    title=f"Channel {channel}",
    xlim=(0, 25),
    ylim=(0, 100),
)
ax.axvline(3, c="r", ymin=0.4, lw=1, alpha=0.5)
ax.axvline(3, c="r", ymax=0.1, lw=1, alpha=0.5)
display(fig)
plt.close(fig)

In [ ]:
channel = 0
slice_max = 0.025
numLines = 20
firstLine = 7
dev_plot = True
dev_name = "dev"

data = df[(df.channel == channel)]
correct = correct_substrate_heating(
    data.height_mV, data.sumV, dev_plot=True, dev_name=f"ch{channel}"
)
pulseV_laser = correct
sum_heights = data.sumV
pulseV_laser_other = sum_heights - pulseV_laser.astype(np.float32)

fig, ax = plt.subplots(figsize=(10, 6), dpi=DPI_VIS)
ax.scatter(
    pulseV_laser,
    data.otherV,
    s=0.3,
    alpha=0.4,
    lw=0,
)
ax.set(
    xlabel="Height [mV]",
    ylabel="Sum [$_\\sum$mV]",
    title=f"Channel {channel}",
    xlim=(0, 25),
    ylim=(0, 100),
)
ax.axvline(3, c="r", ymin=0.4, lw=1, alpha=0.5)
ax.axvline(3, c="r", ymax=0.1, lw=1, alpha=0.5)
display(fig)
plt.close(fig)

In [ ]:
#: TODO Should deskewFFT take all entries to preserve column shape?
freqmaxFFT, maxFFT, angmaxFFT, gradients = deskewFFT(pulseV_laser, pulseV_laser_other)

In [ ]:
1 / freqmaxFFT[np.argmax(maxFFT)]

The FFT works!! We see a clear peak in the gradient testing and the slices do in fact fit to linear approximations of the banding for a second order correction.

In [ ]:
#: Axes flipped to seek flat lines rather than vertical ones
xs = pulseV_laser_other
ys = pulseV_laser - pulseV_laser_other * gradients[np.argmax(maxFFT)]


#: Limits via percentile
xmin, xmax = np.percentile(xs, [5, 95])
ymin, ymax = np.percentile(ys, [10, 80])

#: Angmax may vary from -pi to pi, but its basis is still 2pi
offset = angmaxFFT[np.argmax(maxFFT)] / 2 / np.pi

sliceEdges = np.arange(
    #: Offset is located at the peak heights, slice between peaks with 0.5
    (0.5 + offset) / freqmaxFFT[np.argmax(maxFFT)],
    ymax,
    1 / freqmaxFFT[np.argmax(maxFFT)],
)

#: Cull lines below ymin
sliceEdges = sliceEdges[sliceEdges > ymin]

corrections = np.zeros(len(sliceEdges), dtype=float)

if dev_plot:
    plt.plot(gradients, maxFFT)
    plt.xlabel("Gradient")
    plt.ylabel("Max FFT")
    plt.title(f"Max FFT\ngradient: {gradients[np.argmax(maxFFT)]}")
    display(plt.gcf())
    plt.close()
    # plt.savefig(f"dev/{dev_name}-maxfft.jpeg")
for i, ymini in enumerate(sliceEdges):
    ymaxi = sliceEdges[i + 1] if i + 1 < len(sliceEdges) else ymax
    cutoutx = (pulseV_laser_other < xmax) & (pulseV_laser_other > xmin)
    cutouty = (ymini < ys) & (ymaxi > ys)
    cutout = cutoutx & cutouty
    try:
        poly = np.poly1d(np.polyfit(xs[cutout], ys[cutout], 1))
    except TypeError:
        continue
    if dev_plot:
        plt.plot([xmin, xmax], poly([xmin, xmax]), color="orange")
        pass
#     corrections[i] = poly[1]
if dev_plot:
    plt.scatter(xs, ys, s=0.1, alpha=0.4, lw=0)

    # Find the range where 90% of the data lie in X and Y
    x_min, x_max = np.percentile(xs, [5, 95])
    y_min, y_max = np.percentile(ys, [5, 95])

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    # plt.savefig(f"dev/{dev_name}-slices.jpeg")
    display(plt.gcf())
    plt.close()
avg_correction = np.average(corrections)
true_grad = gradients[np.argmax(maxFFT)] + avg_correction

Turns out it's better to have LOTS of bins and zoom in closely to the low-frequency side. This makes sense but it's weird that the degree of bin granularity isn't what improves FFT resolution or accuracy in detecting periodicity but rather the simple expanse transformed, even if the values in those bins are zero.

In [ ]:
bins = int(1e5)
bestG = 0.021

xs = pulseV_laser - pulseV_laser_other * bestG
ys = pulseV_laser_other

xmin, xmax = np.percentile(xs, [5, 95])
xMin, xMax = np.min(xs), np.max(xs)
range = [xMin, xMax]

h, b = np.histogram(xs, range=range, bins=bins)
fft = np.fft.rfft(h - np.mean(h), norm="ortho")
fig, ax = plt.subplots(figsize=(10, 6), dpi=DPI_VIS)
ax.scatter(xs, ys, s=0.3, alpha=0.4, lw=0)
ax.set(
    xlabel="Height [mV]",
    ylabel="Sum [$_\\sum$mV]",
    title=f"Channel {channel}",
    xlim=range,
    ylim=(0, 100),
)
display(fig)
plt.close(fig)

plt.figure(figsize=(10, 6), dpi=DPI_VIS)
plt.step(b[:-1], h, where="post", lw=0.05)
plt.gca().set(xlabel="Height corrected", ylabel="Counts", xlim=range, yscale="log")
display(plt.gcf())
plt.close()

fftfreq = np.fft.rfftfreq(bins, d=np.diff(range) / bins)
plt.figure(figsize=(10, 6), dpi=DPI_VIS)
plt.plot(fftfreq, np.abs(fft))
plt.gca().set(yscale="log", xlim=(-5, 20))
display(plt.gcf())
plt.close()

In [ ]:
plt.close("all")
pulseHeight = pulseV_laser
otherHeight = pulseV_laser_other
gradientRange = (-0.040, 0.040, 0.0005)
mVmin = 1.1

xmin, xmax = np.percentile(pulseHeight, [5, 95])

# fmt: off
gradients = np.arange(*gradientRange, dtype=np.float32)
maxFFT    = np.zeros_like(gradients,  dtype=np.float32)
angmaxFFT = np.zeros_like(gradients,  dtype=np.float32)
freqmaxFFT = np.zeros_like(gradients,  dtype=np.float32)
# fmt: on

plt.figure(figsize=(10, 6), dpi=DPI_VIS)
for i, grad in list(enumerate(gradients)):
    xs = pulseHeight - otherHeight * grad
    range = [np.min(xs), np.max(xs)]
    h, _ = np.histogram(xs, range=range, bins=bins)
    fft = np.fft.rfft(h - np.mean(h), norm="ortho")
    #: Calculate x-axis (frequency in per mV (pulse height) units)
    #: d is units of pulse height (mV) per bin
    fftfreq = np.fft.rfftfreq(bins, d=np.diff(range) / bins)
    plt.plot(fftfreq, np.abs(fft))
    plt.gca().set(yscale="log")
    argmaxFFT = argmax_part(np.abs(fft), fftfreq > mVmin)
    freqmaxFFT[i] = fftfreq[argmaxFFT]
    maxFFT[i] = np.abs(fft)[argmaxFFT]
    angmaxFFT[i] = np.angle(fft[argmaxFFT])
plt.gca().set(xlabel="Frequency", ylabel="FFT", xlim=(0, 3))
plt.axhline(max(maxFFT))
plt.axvline(freqmaxFFT[np.argmax(maxFFT)])
display(plt.gcf())
plt.close()

A close-up of the FFT near the periodicity bump using the correct units of frequency on the x-axis. This gets us the ~1.6 value we expect by eye looking at the scatter above.

In [ ]:
bins = int(1e5)
bestG = 0.021
plt.figure(figsize=(10, 6), dpi=DPI_VIS)
for g in np.arange(0, 0.03, 0.001):
    h, _ = np.histogram(
        pulseV_laser - pulseV_laser_other * g, range=[xmin, xmax], bins=bins
    )
    fft = np.fft.rfft(h - np.mean(h), norm="ortho")

    #: FFT frequency units in inverse binwidth (mV)
    maxV = np.max(pulseV_laser - pulseV_laser_other * g)
    fftfreq = np.fft.rfftfreq(bins, d=(xmax - xmin) / bins)
    plt.plot(fftfreq, np.abs(fft))
    plt.gca().set(yscale="log")
plt.gca().set(xlabel="Frequency", ylabel="FFT", xlim=(0, 5))
display(plt.gcf())
plt.close()

## Finished Debugging DeskewFFT

The old version was getting lucky assuming that the bin number of the frequency maximum corresponded to the inverse frequency in mV units. Likely it could do this because the range of the FFT was from 0 to 1. For real unit FFT, one needs to use `np.fftfreq` and understand that the "frequency" is in units of pulse height (here mV).

I updated `deskewFFT` to pass the freqmaxFFT rather than the argmaxFFT which is in units of mV (generally pulse height). The minimum in the FFT where we look for periodicity is also now in units of the pulse height. This should generally be consistent across detectors so long as we understand what the typical pulse height of a laser signal is for that detector. At our resolutions this is greater than a single mV depending on the responsivity.

Before COMITTING, cleaning up, and moving on, let me validate that deskewFFT works.

In [ ]:
def argmax_part(array, condition):
    """Return the max position of an array given a condition filters the array.
    The position is in the full array despite the condition possibly being a subset.
    """
    #: Where returns indices where the condition is true, then argmax finds the max index in the same subset
    return np.where(condition)[0][np.argmax(array[condition])]

In [ ]:
xs = pulseV_laser_other
ys = pulseV_laser - pulseV_laser_other * bestG
freqmaxFFT = argmax_part(np.abs(fft), fftfreq > 0.3)  #: The low Freq bump
# argmaxFFT = argmax_part(np.abs(fft), fftfreq > 1) #: The probably right one
angmaxFFT = np.angle(fft[freqmaxFFT])

#: Limits via percentile
xmin, xmax = np.percentile(xs, [5, 95])
ymin, ymax = np.percentile(ys, [10, 80])

#: Angmax may vary from -pi to pi, but its basis is still 2pi
offset = angmaxFFT / 2 / np.pi

sliceEdges = np.arange(
    #: Offset is located at the peak heights, slice between peaks with 0.5
    (0.5 + offset) / fftfreq[freqmaxFFT],
    ymax,
    1 / fftfreq[freqmaxFFT],
)

#: Cull lines below ymin
sliceEdges = sliceEdges[sliceEdges > ymin]

fig, ax = plt.subplots(figsize=(6, 6), dpi=DPI_VIS)
ax.scatter(xs, ys, s=0.3, alpha=0.4, lw=0)
ax.set(
    ylabel="Height [mV]",
    xlabel="Other [$_\\sum$mV]",
    title=f"Channel {channel}",
    xlim=(0, 100),
    ylim=(0, 15),
)
for i, ymini in enumerate(sliceEdges):
    ax.axhline(y=ymini, xmin=0.5, color="orange", alpha=0.4)
    pass
display(fig)
plt.close(fig)

In [ ]:
deskewFFT(pulseV_laser, pulseV_laser_other)

### (On Hold) Periodicity Discovery

Background won't be periodic like laser data will, so we should be able to discover laser events based on periodicity.
But, I can't make sense of the FFT

In [ ]:
df.multiplicity.max()

In [ ]:
# Compute the time differences between consecutive events
event_times = df[df.multiplicity > 5].time
time_diffs = np.diff(event_times)
# time_diffs = time_diffs[time_diffs.nonzero()[0]]
N = len(time_diffs)

# Apply FFT to the time differences
yf = np.fft.fft(time_diffs - np.mean(time_diffs), norm="ortho")
xf = np.fft.fftfreq(N, d=2e-8)

# Find the most significant frequency
idx = np.argmax(np.abs(yf[: N // 4]))
dominant_freq = xf[idx]

# Calculate the most periodic time interval
period = 1 / dominant_freq

print(f"The most periodic time interval is approximately {period*1e9} nanoseconds.")

In [ ]:
xf[idx]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(
    xf,
    np.abs(yf),
    # s=1,
    # lw=0,
)
ax.scatter([xf[idx]], [np.abs(yf[idx])], c="r", s=10, zorder=2)
ax.set_yscale("log")
ax.set_xlim(0, 1e7)
ax.set_ylim(1e-6, 1)

#### Modulo

Event Time vs Modulo_100 $\pm$ something

In [ ]:
#: Event Time vs Modulo 100 ms Plus/Minus Steps up to 50 ms
# df["ig_laser"] =

fig, ax = plt.subplots(figsize=(10, 6), dpi=250)
data = df[(df.channel == channel) & (df.fname == df.fname.unique()[0])]
ax.scatter(data.time, data.time % 0.001, s=0.1, lw=0)
# ax.plot(df.time, (df.time % 0.1).rolling(1000).median(), lw=1, c="r")


# fig, ax = plt.subplots(figsize=(10, 6))
# df["ig_laser"] = np.abs(
#     (df.time % 0.05) - (df.time % 0.05).rolling(1000).median()
# ) < 0.005 * np.mean(df.time % 0.05)
# ax.scatter(df.time[df.ig_laser], df.time[df.ig_laser] % 0.05, s=0.1, lw=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
df.height[df.ig_laser].hist(bins=1000, histtype="step", lw=1, color="r", ax=ax)
ax.set(yscale="log")

fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
h, e = np.histogram(df.height_mV[~df.ig_laser], bins=np.arange(0, 25, 0.1))
ax.step(e[:-1], h, lw=1, color="b")
ax.set(yscale="log")

fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
ax.scatter(df[~df.ig_laser].time, df[~df.ig_laser].height_mV, s=0.1, lw=0, color="b")
ax.set(yscale="log", xlim=(0, 50e3))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
for file in df.fname.unique():
    d = df[(df.fname == file) & (df.ig_laser)]
    ax.scatter(d.height_mV, d.otherV, s=0.1, lw=0, alpha=0.1, label=file)
ax.legend(
    loc="upper right",
    fontsize=8,
    # title="File",
    # title_fontsize=8,
    # shadow=True,
    # fancybox=True,
    # frameon=True,
    # framealpha=0.5,
    # edgecolor="black",
    # facecolor="white",
    ncol=2,
    markerscale=10,
    scatterpoints=10,
    # handletextpad=0.1,
    # handlelength=0.5,
    # handleheight=0.5,
    # borderpad=0.1,
    # labelspacing=0.1,
    # columnspacing=0.1,
    # numpoints=1,
    # mode="expand",
    bbox_to_anchor=(1.05, 1),
    # bbox_transform=None,
    # handler_map=None,
)

# Exploring Other Files

Maybe I just have bad data in the file I'm working with. Let's explore all the files.

In [ ]:
{
    d: sum([os.path.getsize(x) for x in glob.glob(os.path.join(d, "*.bin"))])
    / 1024
    / 1024
    / 1024
    for d in set([os.path.dirname(f) for f in files])
}

In [ ]:
df = pd.DataFrame()
total = 0
for file in [file for file in files if "20240806" in os.path.dirname(file)]:
    dfi, header, _ = load_and_process(file, multithread=True)
    print(f"Size: {os.path.getsize(file) / 1024 / 1024 / 1024} GB")
    if len(dfi) == 0:
        print(f"Skipping {file}")
        continue
    dfi["ig_laser"] = np.abs(
        (dfi.time % 0.1)
        - (dfi.time % 0.1).rolling(min([1000, int(0.01 * len(dfi))])).median()
    ) < 0.01 * np.mean(dfi.time % 0.1)
    dfi["fname"] = os.path.basename(file)
    total += os.path.getsize(file)
    print(f"Loaded Size: {total / 1024 / 1024 / 1024} GB")
    df = pd.concat([df, dfi])

In [ ]:
for file in files:
    df, _, _ = load_and_process(file, multithread=False)
    if len(df) == 0:
        print(f"Skipping {file}")
        continue
    df["ig_laser"] = np.abs(
        (df.time % 0.1)
        - (df.time % 0.1).rolling(min([1000, int(0.01 * len(df))])).median()
    ) < 0.01 * np.mean(df.time % 0.1)
    fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
    df[df.ig_laser].plot(x="height_mV", y="otherV", kind="scatter", s=0.1, lw=0, ax=ax)
    plt.pause(0.1)
    plt.close()
    if len(df[df.ig_laser]) > 0:
        fig, ax = plt.subplots(figsize=(10, 6), dpi=200)
        [
            ax.plot(trc, alpha=0.1)
            for trc in df.trace[df.ig_laser].sample(
                n=min([50, int(0.01 * len(df[df.ig_laser]))])
            )
        ]
        plt.pause(0.1)
        plt.close()

#### Export

In [ ]:
for channel in df.channel.unique():
    key = f"data_channel{channel}"
    df[df.channel == channel].to_hdf(
        f"../../out/{os.path.basename(dev_file)}.h5", key=key
    )
    header["channel"] = channel
    pd.Series(header).to_hdf(
        f"../../out/{os.path.basename(dev_file)}.h5", key=f"meta{key}", mode="w"
    )

## Run Calibration Script

First try tagging mode then try calibration mode.

May need `-e 4` for setting calibration energy to new mode?

In [ ]:
%run "../scripts/laser_heating_calibration.py" -d "../../out/listmode_11_card_0_UTC_2024-07-23_11.14.23_chunk76.bin.h5" -m tagging -e 4 -c 0

#### Dev: Did It Work?

In [ ]:
pd.read_hdf(f"../../out/{os.path.basename(dev_file)}.h5", key=f"data_channel0")

# --

In [ ]:
for channel in sorted(df.channel.unique()):
    fig, ax = plt.subplots(figsize=(8, 4.5), dpi=100)

    h, e = np.histogram(df[df.channel == channel].height, bins=1000)

    ax.step(e[:-1], h)

    ax.set_yscale("log")
    ax.legend()
    ax.set_title(f"Multiplicity (ch {channel})\n{dev_file.split('/')[-4:-2]}")
    ax.set_xlabel("Height")
    # ax.set_xlim(0, 500)
    ax.set_ylabel("Counts")
    plt.show()
    plt.close()

In [ ]:
for channel in sorted(df.channel.unique()):
    fig, ax = plt.subplots(figsize=(8, 4.5), dpi=100)

    h, e = np.histogram(df[df.channel == channel].height, bins=1000)

    ax.step(e[:-1], h)

    for mult in sorted(df[df.channel == channel].multiplicity.unique()):
        if mult == 1:
            continue
        h, _ = np.histogram(
            df[df.multiplicity == mult][df.channel == channel].height, bins=e
        )
        ax.step(e[:-1], h, label=f"Multiplicity {mult}")

    ax.set_yscale("log")
    ax.legend()
    ax.set_title(f"Multiplicity (ch {channel})\n{dev_file.split('/')[-4:-2]}")
    ax.set_xlabel("Height")
    ax.set_xlim(0, 500)
    ax.set_ylabel("Counts")
    plt.show()
    plt.close()

In [ ]:
channel = 28

micro_df = df[df["channel"] == channel]
max_sumV = max(micro_df["sumV"])
filt = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 2)]
data = filt["height_mV"]

In [ ]:
plt.figure(figsize=(9, 5))

nBins = 300
hRange = (0, 90)

counts, edges = np.histogram(data, bins=nBins, range=hRange)
max_bin_index = np.argmax(counts)
max_bin_value = edges[max_bin_index]

counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")

bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

plt.title(f"Peak Amplitude Voltage Spectrum, Channel {channel}")
plt.xlabel("Millivolts")
plt.ylabel("Counts")
plt.tight_layout()
plt.xlim(hRange)

plt.show()
plt.close()

In [ ]:
for channel in df1["channel"].unique():

    micro_df_a = df1[df1["channel"] == channel]
    micro_df_b = df2[df2["channel"] == channel]
    micro_df_c = df3[df3["channel"] == channel]
    micro_df_d = df4[df4["channel"] == channel]

    micro_df = pd.concat(
        [micro_df_a, micro_df_b, micro_df_c, micro_df_d], ignore_index=True
    )

    max_sumV = max(micro_df["sumV"])
    filt = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 2)]
    data = filt["height_mV"]

    plt.figure(figsize=(9, 5))

    nBins = 500
    hRange = (0, 100)

    counts, edges = np.histogram(data, bins=nBins, range=hRange)
    max_bin_index = np.argmax(counts)
    max_bin_value = edges[max_bin_index]

    counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    plt.title(f"Peak Amplitude Voltage Spectrum, Channel {channel}")
    plt.xlabel("Millivolts")
    plt.ylabel("Counts [0.2/mV]")
    plt.tight_layout()
    plt.xlim(hRange)

    plt.show()
    plt.close()

In [ ]:
df1, h1, ktpz = load_and_process(files[0])
df1.drop("trace", axis=1, inplace=True)

# Super long overnight processing code

In [ ]:
channel = 28

# save_file = "20241010_processed_uncleaned2.csv"

for i, file in enumerate(files):
    print("/n----------------------------")
    print(f"Processing file {i} of {len(files)}")
    if i == 0:
        print("skipping seed file")
    else:
        df, _, _ = load_and_process(file, known_tpz=ktpz)
        df.drop("trace", axis=1, inplace=True)
        df1 = pd.concat([df1, df], ignore_index=True)

    print(df1)
    # save progress
    #    df1.to_csv(save_file, mode='w', index=False)
    # plot

    micro_df = df1[df1["channel"] == channel]

    max_sumV = max(micro_df["sumV"])
    filt = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 2)]
    data = filt["height_mV"]

    plt.figure(figsize=(9, 5))

    nBins = 500
    hRange = (0, 100)

    counts, edges = np.histogram(data, bins=nBins, range=hRange)
    max_bin_index = np.argmax(counts)
    max_bin_value = edges[max_bin_index]

    counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    plt.title(f"Voltage Spectrum, Channel {channel}, ~{20000*(i+1)} counts")
    plt.xlabel("Voltage [mV]")
    plt.ylabel("Counts [0.2/mV]")
    plt.tight_layout()
    plt.xlim(hRange)

    plt.show()
    plt.close()

In [ ]:
df1 = pd.read_csv("20241010_processed_uncleaned.csv")
df1

In [ ]:
for channel in df1["channel"].unique():

    micro_df = df1[df1["channel"] == channel]

    max_sumV = max(micro_df["sumV"])
    filt = micro_df[
        (micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 3)
    ]  # we dont want multiplicty 2
    filt2 = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] == 1)]
    data = filt["height_mV"]
    data2 = filt2["height_mV"]

    plt.figure(figsize=(9, 5))

    nBins = 500
    hRange = (0, 100)

    counts, edges = np.histogram(data, bins=nBins, range=hRange)
    max_bin_index = np.argmax(counts)
    max_bin_value = edges[max_bin_index]

    counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")
    plt.hist(data2, bins=nBins, range=hRange, histtype="step", color="red")

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    plt.title(f"Peak Amplitude Voltage Spectrum, Channel {channel}")
    plt.xlabel("Millivolts")
    plt.ylabel("[Counts/0.2mV]")
    plt.tight_layout()
    plt.xlim(hRange)

    plt.show()
    plt.close()

In [ ]:
def gaussian(x, A, mu, sigma):
    return A * np.exp(-((x - mu) ** 2) / (2 * sigma**2))


# Function to fit Gaussian to a peak and return centroid and error
def fit_gaussian_to_peak(x_data, y_data, peak_index, window=10):
    # Select a slice around the peak
    start = max(peak_index - window // 2, 0)
    end = min(peak_index + window // 2, len(x_data))
    x_slice = x_data[start:end]
    y_slice = y_data[start:end]

    # Initial guess: A = max(y), mu = x at peak, sigma = small guess
    initial_guesses = [max(y_slice), x_data[peak_index], 1.0]

    try:
        # Fit Gaussian to the data slice
        params, covariance = curve_fit(gaussian, x_slice, y_slice, p0=initial_guesses)
        # Centroid is mu, error on centroid is sqrt(covariance on mu)
        centroid = params[1]
        centroid_error = np.sqrt(covariance[1, 1])

        # Extract sigma and its error
        sigma = params[2]
        sigma_error = np.sqrt(
            covariance[2, 2]
        )  # sigma error is the square root of the diagonal element

        return centroid, centroid_error, sigma, sigma_error, params
    except Exception as e:
        print(f"Error fitting peak at index {peak_index}: {e}")
        return None, None, None, None, None

In [ ]:
channel = 28

micro_df = df1[df1["channel"] == channel]
max_sumV = max(micro_df["sumV"])
filt = micro_df[(micro_df["otherV"] < 10000) & (micro_df["multiplicity"] >= 2)]
data = filt["height_mV"]

plt.figure(figsize=(9, 5))

nBins = 500
hRange = (0, 100)

counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")

bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
peaks, _ = find_peaks(counts, height=0.01, distance=6)

plt.plot(bin_centers[peaks], counts[peaks], "x", color="black", markersize=6)

centroids = []
centroid_errors = []
sigmas = []
sigma_errors = []

for peak_index in peaks:
    centroid, centroid_error, sigma, sigma_error, params = fit_gaussian_to_peak(
        bin_centers, counts, peak_index
    )
    if centroid is not None:
        centroids.append(centroid)
        centroid_errors.append(centroid_error)
        sigmas.append(sigma)
        sigma_errors.append(sigma_error)

        g_data = gaussian(bin_centers, *params)

        plt.plot(bin_centers, g_data, alpha=0.3)
        plt.errorbar(
            centroid,
            np.interp(centroid, bin_centers, counts),
            xerr=error,
            fmt="ro",
            label="Peaks with errors",
            capsize=10,
            alpha=0.3,
        )

# Generate new "fit" laser positions by sampling around centroids
new_centroids = np.random.normal(centroids, centroid_errors)
new_sigmas = np.random.normal(sigmas, sigma_errors)

# Recalibrate with degree 2 polynomial
lowest_photon_number = new_centroids[1] // 3.5
lowest_photon_number = (
    lowest_photon_number
    if np.abs(3.5 * lowest_photon_number - new_centroids[0])
    < (np.abs(3.5 * (lowest_photon_number + 1) - new_centroids[0]))
    else lowest_photon_number + 1
)
photon_energies = [(i + lowest_photon_number) * 3.5 for i in range(len(new_centroids))]
calibration = np.poly1d(np.polyfit(new_centroids, photon_energies, 2))

print(calibration)

shift = calibration(bin_centers)  # apply calibration
plt.plot(shift, counts, color="orange", linewidth=2)  # replot


for i in range(1, 20):
    plt.axvline(i * 3.5, color="black", alpha=0.2)


plt.title(f"Peak Amplitude Voltage Spectrum, Channel {channel}")
plt.xlabel("Voltage [mV]")
plt.ylabel("[Counts/0.2mV]")
plt.tight_layout()
plt.xlim(hRange)

plt.show()
plt.close()

# Multithread testing

In [ ]:
files = glob.glob(os.path.join(directory, subpaths[1] + "/*.bin"))
files

# 6 hour processing time estimate (channel 10 is bad which is why the index errors)

In [ ]:
df1, header, ktpz = load_and_process(files[0], multithread=True)
df1

In [ ]:
df1.drop("trace", axis=1, inplace=True)
df1

# pre-check

In [ ]:
for channel in df1["channel"].unique():

    micro_df = df1[df1["channel"] == channel]

    max_sumV = max(micro_df["sumV"])
    filt = micro_df[
        (micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 3)
    ]  # we dont want multiplicty 2
    filt2 = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] == 1)]
    data = filt["height_mV"]
    data2 = filt2["height_mV"]

    plt.figure(figsize=(9, 5))

    nBins = 500
    hRange = (0, 100)

    counts, edges = np.histogram(data, bins=nBins, range=hRange)
    max_bin_index = np.argmax(counts)
    max_bin_value = edges[max_bin_index]

    counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")
    plt.hist(data2, bins=nBins, range=hRange, histtype="step", color="red")

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    plt.title(f"Peak Amplitude Voltage Spectrum, Channel {channel}")
    plt.xlabel("Millivolts")
    plt.ylabel("[Counts/0.2mV]")
    plt.tight_layout()
    plt.xlim(hRange)

    plt.show()
    plt.close()

In [ ]:
channel = 5  # for previews

x = x / 5  # dont run this

# save_file = "20241012_Photoresist_7_23_processed_cleaning_tagged.csv"

for i, file in enumerate(files):
    print("/n----------------------------")
    print(f"Processing file {i} of {len(files)}")
    if i == 0:
        print("skipping seed file")
    else:
        try:
            df, _, _ = load_and_process(file, known_tpz=ktpz, multithread=True)
            df.drop("trace", axis=1, inplace=True)
            df1 = pd.concat([df1, df], ignore_index=True)
        except Exception as e:
            continue

    print(df1)
    # save progress
    # df1.to_csv(save_file, mode='w', index=False)
    # plot

    # plots for my sanity

    micro_df = df1[df1["channel"] == channel]

    max_sumV = max(micro_df["sumV"])
    filt = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 2)]
    data = filt["height_mV"]
    plt.figure(figsize=(9, 5))

    nBins = 500
    hRange = (0, 100)

    counts, edges = np.histogram(data, bins=nBins, range=hRange)
    max_bin_index = np.argmax(counts)
    max_bin_value = edges[max_bin_index]

    counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    plt.title(f"Voltage Spectrum, Channel {channel}, ~{20000*(i+1)} counts")
    plt.xlabel("Voltage [mV]")
    plt.ylabel("Counts [0.2/mV]")
    plt.tight_layout()
    plt.xlim(hRange)

    plt.show()
    plt.close()

# post output

In [ ]:
df1

In [ ]:
for channel in df1["channel"].unique():

    micro_df = df1[df1["channel"] == channel]

    max_sumV = max(micro_df["sumV"])
    filt = micro_df[
        (micro_df["otherV"] < 2000) & (micro_df["multiplicity"] >= 3)
    ]  # we dont want multiplicty 2
    filt2 = micro_df[(micro_df["otherV"] < 2000) & (micro_df["multiplicity"] == 1)]
    data = filt["height_mV"]
    data2 = filt2["height_mV"]

    plt.figure(figsize=(9, 5))

    nBins = 500
    hRange = (0, 100)

    counts, edges = np.histogram(data, bins=nBins, range=hRange)
    max_bin_index = np.argmax(counts)
    max_bin_value = edges[max_bin_index]

    counts, bin_edges, _ = plt.hist(data, bins=nBins, range=hRange, histtype="step")
    plt.hist(data2, bins=nBins, range=hRange, histtype="step", color="red")

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    plt.title(f"Peak Amplitude Voltage Spectrum, Channel {channel}")
    plt.xlabel("Millivolts")
    plt.ylabel("[Counts/0.2mV]")
    plt.tight_layout()
    plt.xlim(hRange)

    plt.show()
    plt.close()

In [ ]:
df1 = pd.read_csv("20241012_Photoresist_7_23_processed_cleaning_tagged.csv")

In [ ]:
df1

In [ ]:
channel = 0

micro_df = df2[df2["channel"] == channel]
max_sumV = max(micro_df["sumV"])
filt = micro_df[(micro_df["otherV"] < 10000) & (micro_df["multiplicity"] >= 2)]
filt_be7 = micro_df[(micro_df["otherV"] < 10000) & (micro_df["multiplicity"] == 1)]

data = filt["height_mV"]
data_be7 = filt_be7["height_mV"]

plt.figure(figsize=(9, 5))

nBins = 500
hRange = (0, 100)

counts, bin_edges, _ = plt.hist(
    data,
    bins=nBins,
    range=hRange,
    histtype="step",
    label=f"Laser [Counts/0.2mV], M >= 2: {len(data)}",
)
counts_be7, _ = np.histogram(data_be7, bins=nBins, range=hRange)

bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
peaks, _ = find_peaks(counts, height=0.01, distance=6)

plt.plot(bin_centers[peaks], counts[peaks], "x", color="black", markersize=6)

centroids = []
centroid_errors = []
sigmas = []
sigma_errors = []

for peak_index in peaks:
    centroid, centroid_error, sigma, sigma_error, params = fit_gaussian_to_peak(
        bin_centers, counts, peak_index
    )
    if centroid is not None:
        centroids.append(centroid)
        centroid_errors.append(centroid_error)
        sigmas.append(sigma)
        sigma_errors.append(sigma_error)

        g_data = gaussian(bin_centers, *params)

        plt.plot(bin_centers, g_data, alpha=0.4, linewidth=0.5, linestyle="--")
        # plt.errorbar(centroid, np.interp(centroid, bin_centers, counts), xerr=error, fmt='ro', label="Peaks with errors", capsize=10, alpha=0.3)

# Generate new "fit" laser positions by sampling around centroids

# cuttoff low and highh
low = 1
high = 32

centroids = centroids[low:high]
centroid_errors = centroid_errors[low:high]
sigmas = sigmas[low:high]
sigma_errors = sigma_errors[low:high]

new_centroids = np.random.normal(centroids, centroid_errors)
new_sigmas = np.random.normal(sigmas, sigma_errors)

# Recalibrate with degree 2 polynomial
lowest_photon_number = new_centroids[1] // 3.5
lowest_photon_number = (
    lowest_photon_number
    if np.abs(3.5 * lowest_photon_number - new_centroids[0])
    < (np.abs(3.5 * (lowest_photon_number + 1) - new_centroids[0]))
    else lowest_photon_number + 1
)
photon_energies = [(i + lowest_photon_number) * 3.5 for i in range(len(new_centroids))]
calibration = np.poly1d(np.polyfit(new_centroids, photon_energies, 2))

print(calibration)

shift = calibration(bin_centers)  # apply calibration
plt.step(
    shift,
    counts,
    color="orange",
    linewidth=1,
    label=f"Laser [Counts/0.2eV], M >= 2: {len(data)}",
)  # replot
plt.step(
    shift[30:],
    counts_be7[30:] * 300,
    color="red",
    linewidth=1,
    label=f"Be7 [300*Counts/0.2eV], M == 1: {len(data_be7)}",
)

for i in range(1, 50):
    plt.axvline(i * 3.5, color="grey", alpha=0.2)

plt.axvline(107.92, color="green", linestyle="--", label="107.92 eV")


plt.title(f"Voltage/Energy Spectrum, Channel {channel}")
plt.xlabel("Blue-Voltage[mV], Orange-Energy[eV]")
plt.ylabel("Counts")
plt.tight_layout()
plt.legend()
plt.xlim(0, 175)

plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 5))

plt.step(
    shift,
    counts,
    color="orange",
    linewidth=1,
    label=f"Laser [Counts/0.2eV], M >= 2: {len(data)}",
)  # replot
plt.step(
    shift[15:],
    counts_be7[15:] * 300,
    color="red",
    linewidth=1,
    label=f"Be7 [300*Counts/0.2eV], M == 1: {len(data_be7)}",
)
plt.axvline(107.92, color="green", linestyle="--", label="107.92 eV")

plt.title("Channel 0 Spectra")
plt.xlabel("Energy [eV]")
plt.ylabel("[Counts/0.2eV]")
plt.legend()

plt.xlim(0, 175)

plt.show()
plt.close()

# Be7 Spectrum?

In [ ]:
df1 = pd.read_csv("20241010_processed_uncleaned.csv")
df2 = pd.read_csv("20241012_Photoresist_7_23_processed_cleaning_tagged.csv")

In [ ]:
df2

In [ ]:
def get_calibration_function(bin_centers, counts_laser, low_cut, high_cut):
    peaks, _ = find_peaks(counts_laser, height=0.01, distance=6)

    # fit gaussians and get errors
    centroids = []
    centroid_errors = []
    sigmas = []
    sigma_errors = []

    for peak_index in peaks:
        centroid, centroid_error, sigma, sigma_error, params = fit_gaussian_to_peak(
            bin_centers, counts_laser, peak_index
        )
        if centroid is not None:
            centroids.append(centroid)
            centroid_errors.append(centroid_error)
            sigmas.append(sigma)
            sigma_errors.append(sigma_error)

    centroids = centroids[low_cut:high_cut]
    centroid_errors = centroid_errors[low_cut:high_cut]
    sigmas = sigmas[low_cut:high_cut]
    sigma_errors = sigma_errors[low_cut:high_cut]

    new_centroids = np.random.normal(centroids, centroid_errors)
    new_sigmas = np.random.normal(sigmas, sigma_errors)

    # Recalibrate with degree 2 polynomial
    lowest_photon_number = new_centroids[1] // 3.5
    lowest_photon_number = (
        lowest_photon_number
        if np.abs(3.5 * lowest_photon_number - new_centroids[0])
        < (np.abs(3.5 * (lowest_photon_number + 1) - new_centroids[0]))
        else lowest_photon_number + 1
    )
    photon_energies = [
        (i + lowest_photon_number) * 3.5 for i in range(len(new_centroids))
    ]
    calibration = np.poly1d(np.polyfit(new_centroids, photon_energies, 2))

    return calibration


def cutoff_helper_plot(df, channel, low_cut=5, high_cut=15):
    # generates mV spectrum to visually help choose the peak cutoffs for calibration
    micro_df = df[df["channel"] == channel]
    filt_laser = micro_df[
        (micro_df["otherV"] < 10000) & (micro_df["multiplicity"] >= 2)
    ]
    data_laser = filt_laser["height_mV"]

    nBins = 500
    hRange = (0, 100)

    counts_laser, bin_edges = np.histogram(data_laser, bins=nBins, range=hRange)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    peaks, _ = find_peaks(counts_laser, height=0.01, distance=6)
    calibration = get_calibration_function(bin_centers, counts_laser, low_cut, high_cut)

    fig, ax = plt.subplots(2, 1, figsize=(9, 6))

    ax[0].step(bin_centers, counts_laser, linewidth=0.5, color="blue")
    for i, peak in enumerate(peaks):
        if i == low_cut or i == high_cut:
            ax[0].axvline(bin_centers[peak], color="red")
        ax[0].step(bin_centers[peak], counts_laser[peak], "ro", markersize=1)
        ax[0].text(bin_centers[peak] - 1, counts_laser[peak] + 100, f"{i}", fontsize=6)

    ax[1].step(calibration(bin_centers), counts_laser, linewidth=0.5, color="orange")
    ax[1].step(
        calibration(bin_centers) + 3.5 * (low_cut),
        counts_laser,
        linewidth=0.5,
        color="green",
    )
    for i in range(1, 50):
        ax[1].axvline(i * 3.5, color="grey", alpha=0.2)
    ax[1].set_xlim(-20, 100)  # to show offset

    ax[0].set_title(f"Channel {channel}, cutoffs=({low_cut}, {high_cut})")
    ax[0].set_ylabel("[Counts/0.2mV]")
    ax[0].set_xlabel("Voltage [mV]")

    ax[1].set_ylabel("[Counts/0.2eV]")
    ax[1].set_xlabel("Energy [eV]")

    plt.show()
    plt.close()


def calibrate_voltage_to_energy(df, channel, low_cut, high_cut, debug_plot=False):

    if debug_plot:
        plt.figure(figsize=(9, 5))

    micro_df = df[df["channel"] == channel]
    filt_laser = micro_df[
        (micro_df["otherV"] < 10000) & (micro_df["multiplicity"] >= 2)
    ]
    filt_be7 = micro_df[(micro_df["otherV"] < 10000) & (micro_df["multiplicity"] == 1)]

    data_laser = filt_laser["height_mV"]
    data_be7 = filt_be7["height_mV"]

    nBins = 500
    hRange = (0, 100)

    counts_laser, bin_edges = np.histogram(data_laser, bins=nBins, range=hRange)
    counts_be7, _ = np.histogram(data_be7, bins=nBins, range=hRange)

    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    calibration = get_calibration_function(bin_centers, counts_laser, low_cut, high_cut)

    if debug_plot:
        print(calibration)
        shift = calibration(bin_centers)
        plt.step(
            shift, counts_laser, color="orange", linewidth=1, label=f"Laser, M >= 2"
        )
        plt.step(
            shift[30:],
            counts_be7[30:] * 300,
            color="red",
            linewidth=1,
            label=f"Be7, M == 1",
        )

        # Draw calibration lines
        for i in range(1, 50):
            plt.axvline(i * 3.5, color="grey", alpha=0.2)

        plt.axvline(107.92, color="green", linestyle="--", label="107.92 eV")
        plt.title(f"Voltage/Energy Spectrum, Channel {channel}")
        plt.xlabel("Energy[eV]")
        plt.ylabel("[Counts/0.2eV]")
        plt.tight_layout()
        plt.legend()
        plt.xlim(0, 175)
        plt.show()
        plt.close()

    return calibration, bin_centers, counts_laser, counts_be7

In [ ]:
day1_channels = [0, 2, 3, 4, 5, 8, 9, 10, 13, 14, 17, 18, 19, 20, 21, 26, 28]  # 7/22
d1_cutoffs = [
    (0, 15),
    (2, 9),
    (2, 10),
    (3, 12),
    (4, 15),
    (4, 15),
    (5, 12),
    (5, 14),
    (4, 9),
    (5, 23),
    (4, 12),
    (4, 12),
    (5, 15),
    (5, 15),
    (3, 10),
    (2, 9),
    (5, 19),
]

day2_channels = [0, 3, 5, 8, 9, 11, 17, 19, 20, 21]  # 7/23

channel = 2
cuts = d1_cutoffs[channel]
cutoff_helper_plot(df1, channel, low_cut=cuts[0], high_cut=cuts[1])

In [ ]:
calibration1, bins1, laser1, be1 = calibrate_voltage_to_energy(df1, 0, 4, 15)
calibration2, bins2, laser2, be2 = calibrate_voltage_to_energy(
    df1, 2, 2, 9, debug_plot=True
)

plt.figure(figsize=(10, 6))

plt.step(calibration1(bins1) + 1 * 3.5, laser1)
plt.step(calibration2(bins2) + 2 * 3.5, laser2)

for i in range(1, 50):
    plt.axvline(i * 3.5, color="grey", alpha=0.2)

plt.xlim(-10, 100)
plt.show()
plt.close()

## Pretty Plot For Thesis

In [ ]:
channel = 0

micro_df = df2[df2["channel"] == channel]
max_sumV = max(micro_df["sumV"])
filt = micro_df[(micro_df["otherV"] < 10000) & (micro_df["multiplicity"] >= 2)]
filt_be7 = micro_df[(micro_df["otherV"] < 10000) & (micro_df["multiplicity"] == 1)]

data = filt["height_mV"]
data_be7 = filt_be7["height_mV"]

plt.figure(figsize=(9, 5), dpi=400)

nBins = 500
hRange = (0, 100)

counts, bin_edges = np.histogram(data, bins=nBins, range=hRange)
counts_be7, _ = np.histogram(data_be7, bins=nBins, range=hRange)


bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
peaks, _ = find_peaks(counts, height=0.01, distance=6)

# plt.plot(bin_centers[peaks], counts[peaks], 'x', color='black', markersize=6)

centroids = []
centroid_errors = []
sigmas = []
sigma_errors = []

for peak_index in peaks:
    centroid, centroid_error, sigma, sigma_error, params = fit_gaussian_to_peak(
        bin_centers, counts, peak_index
    )
    if centroid is not None:
        centroids.append(centroid)
        centroid_errors.append(centroid_error)
        sigmas.append(sigma)
        sigma_errors.append(sigma_error)

        # g_data = gaussian(bin_centers, *params)

        # plt.plot(bin_centers, g_data, alpha=0.4, linewidth=0.5, linestyle='--')
        # plt.errorbar(centroid, np.interp(centroid, bin_centers, counts), xerr=error, fmt='ro', label="Peaks with errors", capsize=10, alpha=0.3)

# Generate new "fit" laser positions by sampling around centroids

# cuttoff low and highh
low = 1
high = 32

centroids = centroids[low:high]
centroid_errors = centroid_errors[low:high]
sigmas = sigmas[low:high]
sigma_errors = sigma_errors[low:high]

new_centroids = np.random.normal(centroids, centroid_errors)
new_sigmas = np.random.normal(sigmas, sigma_errors)

# Recalibrate with degree 2 polynomial
lowest_photon_number = new_centroids[1] // 3.5
lowest_photon_number = (
    lowest_photon_number
    if np.abs(3.5 * lowest_photon_number - new_centroids[0])
    < (np.abs(3.5 * (lowest_photon_number + 1) - new_centroids[0]))
    else lowest_photon_number + 1
)
photon_energies = [(i + lowest_photon_number) * 3.5 for i in range(len(new_centroids))]
calibration = np.poly1d(np.polyfit(new_centroids, photon_energies, 2))

print(calibration)

shift = calibration(bin_centers)  # apply calibration
plt.step(
    bin_centers,
    counts,
    color="blue",
    linewidth=1,
    label=f"Laser [Counts/0.2mV], total counts: {len(data)}",
    alpha=0.2,
)
plt.step(
    shift + 3.5,
    counts,
    color="black",
    linewidth=1,
    label=f"Laser [Counts/0.2eV], total counts: {len(data)}",
)  # replot
plt.step(
    shift + 3.5,
    counts_be7 * 200,
    color="red",
    linewidth=1,
    label=f"Be7 [200x Counts/0.2eV], total counts: {len(data_be7)}",
)

for i in range(1, 52):
    plt.axvline(i * 3.5, color="grey", alpha=0.2)

plt.axvline(
    107.92,
    color="green",
    linestyle="--",
    linewidth=1,
    label="Expected K-GS Peak (107.92 eV)",
)

plt.xlabel("Energy [eV] (Black, Red), Voltage [mV] (Blue)")
plt.ylabel("Counts")
plt.tight_layout()
plt.legend()
plt.xlim(0, 180)
plt.ylim(0, 10500)

# plt.savefig("CalibratePhotoresistLaserSpectra.png", dpi=400)


plt.show()
plt.close()